In [2]:
from pyvesc import VESC
from time import sleep

class SpeedController :
    def __init__ (self, vescpath) :
        self.rpmobj = 0
        self.k = 0.001
        self.rpml = list()
        self.vesc = VESC(serial_port=vescpath)
        self.current_duty_cycle = 0

    def set_duty_cycle(self, val):
        if val>0.5:
            val=0.5
        if val<-0.5:
            val=-0.5
        self.vesc.set_duty_cycle(val)
    
    def asserv(self, rpm):
        err = self.rpmobj - rpm
        delta = self.k * err
        return(delta)
    
    def cycle(self):

        try:
            rpm = -self.vesc.get_rpm()
        except AttributeError:
            return

        self.current_duty_cycle = self.current_duty_cycle+self.asserv(rpm)  
        self.set_duty_cycle(self.current_duty_cycle)
        sleep(0.01)

In [3]:
a = SpeedController(vescpath='/dev/serial/by-path/pci-0000:00:14.0-usb-0:7.2:1.0')

In [4]:
a.rpmobj = 50

In [7]:
if n in range(5) :
    a.cycle()

NameError: name 'n' is not defined

NameError: name 'set_duty_cycle' is not defined